In [4]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient("localhost", 27017)
client.drop_database("userData")

db = client.userData
with open("people.json") as f:
    file_data = json.load(f)
db.users.insert_many(file_data)


In [5]:
for x in db.users.aggregate(
    [
        {
            "$project": {
                "_id": 0,
                "location": {
                    "type": "Point",
                    "coordinates": [
                        {
                            "$convert": {
                                "input": "$location.coordinates.longitude",
                                "to": "double",
                                "onError": 0.0,
                                "onNull": 0.0,
                            }
                        },
                        {
                            "$convert": {
                                "input": "$location.coordinates.latitude",
                                "to": "double",
                                "onError": 0.0,
                                "onNull": 0.0,
                            }
                        },
                    ],
                },
            }
        },
        {"$limit": 5},
    ]
):
    pprint(x)


{'location': {'coordinates': [-31.0208, -29.8113], 'type': 'Point'}}
{'location': {'coordinates': [-154.6037, -29.6721], 'type': 'Point'}}
{'location': {'coordinates': [-70.2264, 76.4507], 'type': 'Point'}}
{'location': {'coordinates': [168.9462, -22.5329], 'type': 'Point'}}
{'location': {'coordinates': [-54.1364, -86.1268], 'type': 'Point'}}


In [6]:
client.close()
